In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Caricamento del file Excel
# Caricamento del file Excel dal percorso corretto
df = pd.read_excel(r"C:\Users\ebest\Desktop\df_gpp_lavoratori_xlsx.xlsx", sheet_name='Sheet1')
df.head()

,anno,genere,regione,codsettore,descrizione_settore,ggp,numero_lavoratori
0,2015,F,Abruzzo,01.11.10,"AGRICOLTURA, SILVICOLTURA E PESCA",PROFESSIONI NON QUALIFICATE,1
1,2015,F,Abruzzo,01.11.10,"AGRICOLTURA, SILVICOLTURA E PESCA",PROFESSIONI TECNICHE,2
2,2015,F,Abruzzo,01.19.10,"AGRICOLTURA, SILVICOLTURA E PESCA",PROFESSIONI NON QUALIFICATE,1
3,2015,F,Abruzzo,01.50.00,"AGRICOLTURA, SILVICOLTURA E PESCA",PROFESSIONI NON QUALIFICATE,1
4,2015,F,Abruzzo,01.50.00,"AGRICOLTURA, SILVICOLTURA E PESCA",PROFESSIONI QUALIFICATE NELLE ATTIVITA¿ COMMER...,1


In [6]:
import pandas as pd
import matplotlib.pyplot as plt

# Caricamento del file
df = pd.read_excel(r"C:\Users\ebest\Desktop\df_gpp_lavoratori_xlsx.xlsx", sheet_name='Sheet1')

# Categorie GPG 1 e 2
gpg_categorie = [
    'LEGISLATORI, IMPRENDITORI E ALTA DIRIGENZA',
    'PROFESSIONI INTELLETTUALI, SCIENTIFICHE E DI ELEVATA SPECIALIZZAZIONE'
]

# Filtro per GPG 1 e 2
df_filtered = df[df['ggp'].isin(gpg_categorie)].copy()

# Conversione in numerico
df_filtered['numero_lavoratori'] = pd.to_numeric(df_filtered['numero_lavoratori'], errors='coerce')

# Funzione per il grafico
def plot_occupazione_nazionale(data):
    grouped = data.groupby(['descrizione_settore', 'genere'])['numero_lavoratori'].sum().reset_index()
    pivot = grouped.pivot(index='descrizione_settore', columns='genere', values='numero_lavoratori').fillna(0)
    pivot['Totale'] = pivot.sum(axis=1)
    pivot = pivot.sort_values(by='Totale', ascending=False).drop(columns='Totale')
    
    pivot.plot(kind='bar', stacked=False, figsize=(14, 7))
    plt.title("Occupazione maschile e femminile a livello nazionale (GPG 1 e 2)")
    plt.ylabel("Numero di lavoratori")
    plt.xlabel("Settore industriale")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.legend(title="Genere")
    plt.grid(True)
    plt.show()



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Caricamento dati
df = pd.read_excel(r"C:\Users\ebest\Desktop\df_gpp_lavoratori_xlsx.xlsx", sheet_name='Sheet1')

# Selezione GPG 1 e 2
gpg_categorie = [
    'LEGISLATORI, IMPRENDITORI E ALTA DIRIGENZA',
    'PROFESSIONI INTELLETTUALI, SCIENTIFICHE E DI ELEVATA SPECIALIZZAZIONE'
]
df = df[df['ggp'].isin(gpg_categorie)].copy()
df['numero_lavoratori'] = pd.to_numeric(df['numero_lavoratori'], errors='coerce')

# Lista dei settori unici
settori_unici = df['descrizione_settore'].dropna().unique().tolist()

# Funzione interattiva
def aggiorna_grafico(settore):
    dati = df[df['descrizione_settore'] == settore]
    
    if dati.empty:
        print("Nessun dato disponibile per il settore selezionato.")
        return

    grouped = dati.groupby('genere')['numero_lavoratori'].sum()
    totale = grouped.sum()
    percentuali = (grouped / totale * 100).round(2)

    # Plot
    colors = {'F': '#FF6F61', 'M': '#4A90E2'}
    percentuali.plot(kind='bar', color=[colors.get(g, '#999') for g in percentuali.index], figsize=(6,4))

    plt.title(f"Distribuzione % maschi e femmine\n{settore} (GPG 1 e 2)", fontsize=14, weight='bold')
    plt.ylabel("Percentuale", fontsize=12)
    plt.ylim(0, 100)
    plt.xticks(rotation=0, fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    
    for i, v in enumerate(percentuali):
        plt.text(i, v + 2, f'{v}%', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.show()

# Widget interattivo
interact(aggiorna_grafico, settore=widgets.Dropdown(options=settori_unici, description='Settore:'))




interactive(children=(Dropdown(description='Settore:', options=('ATTIVITÀ MANIFATTURIERE', 'COSTRUZIONI', "COM…

<function __main__.aggiorna_grafico(settore)>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Caricamento dati
df = pd.read_excel(r"C:\Users\ebest\Desktop\df_gpp_lavoratori_xlsx.xlsx", sheet_name='Sheet1')

# Selezione GPG 1 e 2
gpg_categorie = [
    'LEGISLATORI, IMPRENDITORI E ALTA DIRIGENZA',
    'PROFESSIONI INTELLETTUALI, SCIENTIFICHE E DI ELEVATA SPECIALIZZAZIONE'
]
df = df[df['ggp'].isin(gpg_categorie)].copy()
df['numero_lavoratori'] = pd.to_numeric(df['numero_lavoratori'], errors='coerce')

# Liste uniche per widget
settori_unici = sorted(df['descrizione_settore'].dropna().unique())
regioni_uniche = ['Tutte'] + sorted(df['regione'].dropna().unique())

# Funzione interattiva
def aggiorna_grafico(settore, regione):
    dati = df[df['descrizione_settore'] == settore]

    if regione != 'Tutte':
        dati = dati[dati['regione'] == regione]

    if dati.empty:
        print("Nessun dato disponibile per i filtri selezionati.")
        return

    grouped = dati.groupby('genere')['numero_lavoratori'].sum()
    totale = grouped.sum()
    percentuali = (grouped / totale * 100).round(2)

    # Plot
    colors = {'F': '#FF6F61', 'M': '#4A90E2'}
    percentuali.plot(kind='bar', color=[colors.get(g, '#999') for g in percentuali.index], figsize=(6,4))

    titolo = f"{settore} - {regione}" if regione != 'Tutte' else settore
    plt.title(f"Distribuzione % maschi e femmine\n{titolo} (GPG 1 e 2)", fontsize=14, weight='bold')
    plt.ylabel("Percentuale", fontsize=12)
    plt.ylim(0, 100)
    plt.xticks(rotation=0, fontsize=11)
    plt.yticks(fontsize=11)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    
    for i, v in enumerate(percentuali):
        plt.text(i, v + 2, f'{v}%', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.show()

# Widget interattivo
interact(
    aggiorna_grafico,
    settore=widgets.Dropdown(options=settori_unici, description='Settore:'),
    regione=widgets.Dropdown(options=regioni_uniche, description='Regione:')
)


interactive(children=(Dropdown(description='Settore:', options=('AGRICOLTURA, SILVICOLTURA E PESCA', 'ALTRE AT…

<function __main__.aggiorna_grafico(settore, regione)>